In [58]:
import os
from trackml.score  import score_event
from sklearn.manifold import TSNE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import least_squares


In [59]:
# estimate helix paramater space
#reference https://www.kaggle.com/c/trackml-particle-identification/discussion/57643
def helix_estimate_param_from_track(xyz):
    
    def residuals_xy(param, x, y):
        x0, y0 = param
        r = np.sqrt((x-x0)**2 + (y-y0)**2)
        d = r - r.mean()
        return d
    
    def residuals_z(param, r, z):
        m2,m1,m0 = param
        zz = m2*r**2 + m1*r + m0
        d = z - zz
        return d
    
    x = xyz[:3,0]
    y = xyz[:3,1]
    z = xyz[:3,2]
    
    param0 = (x.mean(), y.mean())
    res_lsq = least_squares(residuals_xy, param0, loss='soft_l1', f_scale=1.0, args=(x,y))
    x0,y0 = res_lsq.x
    r0 = np.sqrt((x-x0)**2 + (y-y0)**2).mean()
    r = np.sqrt(x**2 + y**2)
    
    if 1:
        param0 = (0,0,0)
        res_lsq = least_squares(residuals_z, param0, args=(r, z))
        m2, m1, m0 = res_lsq.x
        
    if 0:
        # polynomial fit of the degree 2, x**2 - quadradic 
        m2,m1,m0 = np.polyfit(r,z,2)
        
    param = (x0,y0,r0,m1,m2,m0)
    return param

# convert x,y,z to curve space
def helix_search_theta(xyz, param, S=2.5):
    
    x0, y0, r0, m2, m1, m0 = param
    theta0 = np.arctan2(y0, x0)
    #sort by z
    xyz = xyz[np.argsort(np.fabs(xyz[:,2]))]
    xx = xyz[:,0] - x0
    yy = xyz[:,1] - y0
    x = xx*np.cos(-theta0) - yy*np.sin(-theta0)
    y = xx*np.sin(-theta0) + yy*np.cos(-theta0)
    theta = np.arctan(y, -x)
    
    theta_min = 0
    theta_max = theta[-1]
    theta_num = S*len(xyz)*50
    return np.linspace(theta_min, theta_max, theta_num)
    

def helix_generate_track_from_param(param, theta=np.linspace(0, 2*np.pi, 360)):
    x0, y0, r0, m1, m2, m0 = param
    theta0 = np.arctan2(y0, x0)
    xx = -r0*np.cos(theta)
    yy = r0*np.sin(theta)
    x = xx*np.cos(theta0) - yy*np.sin(theta0)+x0
    y = xx*np.sin(theta0) + yy*np.cos(theta0)+y0
    
    r = np.sqrt(x**2 + y**2)
    z = m2*r**2 + m1*r + m0
    
    xyz = np.column_stack([x,y,z])
    return xyz
        
# calculate r/2r0, only for EDA
def calculate_r0_r_ratio(xyz, r0):
    xyz = xyz[np.argsort(np.fabs(xyz[:,2]))]
    x = xyz[:,0]
    y = xyz[:,1]
    r = np.sqrt(x**2 + y**2)
    ratio = 0.5*r/r0
    #print('r:' + str(r))
    return ratio
    
# only for EDA
def study_helix_param(event_id):
    # Change this from 0 - 83xx :number of particles
    track_id = 1
     ## load an event ---
    event_id = event_id

    data_dir  = '../../../input/train_100_events'
    csv_dir = '../../../input/'
    
    particles = pd.read_csv(os.path.join(data_dir, 'event00000%s-particles.csv'%event_id))
    hits      = pd.read_csv(os.path.join(data_dir, 'event00000%s-hits.csv' %event_id))
    truth     = pd.read_csv(os.path.join(data_dir, 'event00000%s-truth.csv'%event_id))
    
    truth = truth.merge(hits,       on=['hit_id'],      how='left')
    truth = truth.merge(particles,  on=['particle_id'], how='left')
    
    #--------------------------------------------------------
    df = truth.copy()
    p = df[['particle_id']].values.astype(np.int64)
    particle_ids = np.unique(p)
    particle_ids = particle_ids[particle_ids!=0]

    print('particle id:'+ str(particle_ids[track_id]))
    t = df.loc[(df.particle_id==4506004809056256)].as_matrix(columns=['x','y','z'])
    t = t[np.argsort(np.fabs(t[:,2]))]

    param = helix_estimate_param_from_track(t)
    theta = helix_search_theta(t, param, S=1)
    x0, y0, r0, m1, m2, m0 = param
    
    ## Visualization only 
    helix = helix_generate_track_from_param(param, theta=theta)
    
    print(r0)
    print(t)
    x = t[:,0]
    y = t[:,1]
    r = np.sqrt(x**2 + y**2)
    print(r)
    
    fig1 = plt.figure(figsize=(8, 8))
    ax1 = fig1.add_subplot(111, projection='3d')

    ax1.plot(helix[:,0], helix[:,1], helix[:,2], '.-', color=[1,0,0], markersize=1)
    # ground truth
    ax1.plot(t[:,0], t[:,1], t[:,2], '.-', color=[0,1,0], markersize=8)
    
    
    ax1.set_title('xyz')
    ax1.set_xlabel('x (millimeters)')
    ax1.set_ylabel('y (millimeters)')
    ax1.set_zlabel('z (millimeters)')
    
    fig2 = plt.figure(figsize=(8, 8))
    ax2 = fig2.add_subplot(111)
    
    ax2.plot(helix[:,0], helix[:,1], '.-', color=[1,0,0], markersize=1)
    # ground truth
    ax2.plot(t[:,0], t[:,1], '.-', color=[0,1,0], markersize=8)
    
    ax2.set_title('XY projection')
    ax2.set_xlabel('x (millimeters)')
    ax2.set_ylabel('y (millimeters)')
    
def generate_helix_radii_list(event_id, data_dir):
    event_id = event_id
    
    particles = pd.read_csv(os.path.join(data_dir, 'event00000%s-particles.csv'%event_id))
    hits      = pd.read_csv(os.path.join(data_dir, 'event00000%s-hits.csv' %event_id))
    truth     = pd.read_csv(os.path.join(data_dir, 'event00000%s-truth.csv'%event_id))
    
    truth = truth.merge(hits,       on=['hit_id'],      how='left')
    truth = truth.merge(particles,  on=['particle_id'], how='left')
    
    df = truth.copy()
    p = df[['particle_id']].values.astype(np.int64)
    particle_ids = np.unique(p)
    r0_ratio_list = []
    r0_list = []
    id_list = []
    particle_ids = particle_ids[particle_ids!=0]
    for index, _ in enumerate(particle_ids):
        t = df.loc[(df.particle_id==id)].as_matrix(columns=['x','y','z'])
        # remove short tracks
        if len(t) < 4:
            continue 
        t = t[np.argsort(np.fabs(t[:,2]))]
        x0, y0, r0, m1, m2, m0  = helix_estimate_param_from_track(t)
        ratio = calculate_r0_r_ratio(t, r0)
        r0_ratio_list.extend(ratio)
        r0_list.append(r0)
    
    return r0_list, r0_ratio_list, id_list

def generate_radii_csv(event_id):
    r0_list = np.sort(r0_list_0)
    r0_list = np.round(r0_list)
    r0_list = np.unique(r0_list)
    df = pd.DataFrame(r0_list)
    df.to_csv('r0_list_%s.csv'%event_id, index=False, header=['r0'])

    
if __name__ == '__main__':
    # Generate an list of helix radii from the train data 
    # You shouldn't generate radii from the validation data set you're using
    r0_list_1, _, _ = generate_helix_radii_list(event_id=1000, data_dir='../../../input/train_1')
    generate_radii_csv(event_id=1000)



/home/nicole/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:86: RuntimeWarning: invalid value encountered in arccos
